In [ ]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 10
expName = "PSI_Site_DLNN_CORENup_RNAfold"
outPath = "Results"
foldName = "folds.pickle"

# modelNames = ["DLNN_3", "DLNN_5"]

epochs = 100
batch_size = 16
shuffle = True
seed = 22

input_data_folder = "Data\\Aziz"

In [ ]:
import os 
from Bio import SeqIO
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score

import math

In [ ]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
##################################################################################
##### define all CUSTOM functions
##################################################################################

def one_hot_encode_dna(sequence):
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "T": 3
    }
    i = 0
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in DNA sequence: '+sequence)
    return seq_encoded

def one_hot_encode_rna(sequence):
    seq_encoded = np.zeros((len(sequence),4))
    dict_nuc = {
        "A": 0,
        "C": 1,
        "G": 2,
        "U": 3
    }
    i = 0
    for single_character in sequence:
        if(single_character.upper() in dict_nuc.keys()):
            seq_encoded[i][dict_nuc[single_character.upper()]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in RNA sequence: '+sequence)
    return seq_encoded

def one_hot_encode_rnafold(sequence):
    seq_encoded = np.zeros((len(sequence),3))
    dict_fold = {
        "(": 0,
        ")": 1,
        ".": 2
    }
    i = 0
    for single_character in sequence:
        if(single_character in dict_fold.keys()):
            seq_encoded[i][dict_fold[single_character]] = 1
            i = i+1
        else:
            raise ValueError('Incorrect character in RNAfold: '+sequence)
    return seq_encoded

In [ ]:
##################################################################################
##### define evaluator functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def build_kfold_multifeature(features_1, features_2, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features_1, labels):
        X1_train, X1_test = features_1[train_index], features_1[test_index]
        X2_train, X2_test = features_2[train_index], features_2[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X1_train": X1_train,
            "X1_test": X1_test,
            "X2_train": X2_train,
            "X2_test": X2_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [ ]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_CORENup_RNAFold(input_seq_len = 21,
                       conv_filters_per_layer_1 = 50, kernel_length_1 = 5, conv_strides_1 = 1, ## 1st Convolutional layer parameters
                       max_pool_width_1 = 2, max_pool_stride_1 = 2, ## 1st Maxpool layer parameters
                       lstm_decode_units = 50, ## LSTM layer parameters
                       conv_filters_per_layer_2 = 50,  kernel_length_2 = 10, conv_strides_2 = 1, ## 2nd Convolutional layer parameters
                       max_pool_width_2 = 2, max_pool_stride_2 = 2, ## 2nd Maxpool layer parameters
                       dense_decode_units = 400, ## Dense layer parameters
                       prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input_shape_seq = (input_seq_len, 4)
    input_shape_fold = (input_seq_len, 3)
    
    ######################################################################################################
    ########  SEQUENCE  ##################################################################################
    ######################################################################################################
    
    input1 = tf.keras.layers.Input(shape=input_shape_seq)

    x1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta)
                               )(input1)
    x1 = tf.keras.layers.Activation('relu')(x1)
    x1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(x1)
    x1 = tf.keras.layers.Dropout(prob)(x1)

    ## LSTM Path

    x2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(x1)
    x2 = tf.keras.layers.Dropout(prob)(x2)
    
    x2 = tf.keras.layers.Flatten()(x2)

    ## Conv Path

    x3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta), padding = 'same')(x1)
    x3 = tf.keras.layers.Activation('relu')(x3)
    x3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(x3)
    x3 = tf.keras.layers.Dropout(prob)(x3)
    
    x3 = tf.keras.layers.Flatten()(x3)
    
    x4 = tf.keras.layers.Concatenate(1)([x2,x3])
    
    ######################################################################################################
    ########  RNA Fold  ##################################################################################
    ######################################################################################################
    
    input2 = tf.keras.layers.Input(shape=input_shape_fold)
    
    f1 = tf.keras.layers.Conv1D(conv_filters_per_layer_1, kernel_length_1,
                                strides = conv_strides_1, kernel_regularizer = tf.keras.regularizers.l2(beta)
                               )(input2)
    f1 = tf.keras.layers.Activation('relu')(f1)
    f1 = tf.keras.layers.MaxPool1D(pool_size = max_pool_width_1, strides = max_pool_stride_1)(f1)
    f1 = tf.keras.layers.Dropout(prob)(f1)

    ## LSTM Path

    f2 = tf.keras.layers.LSTM(lstm_decode_units, return_sequences = True, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(f1)
    f2 = tf.keras.layers.Dropout(prob)(f2)
    
    f2 = tf.keras.layers.Flatten()(f2)

    ## Conv Path

    f3 = tf.keras.layers.Conv1D(conv_filters_per_layer_2, kernel_length_2, strides = conv_strides_2, 
                                kernel_regularizer = tf.keras.regularizers.l2(beta)
                               )(f1)
    f3 = tf.keras.layers.Activation('relu')(f3)
    f3 = tf.keras.layers.MaxPooling1D(pool_size = max_pool_width_2, strides = max_pool_stride_2)(f3)
    f3 = tf.keras.layers.Dropout(prob)(f3)
    
    f3 = tf.keras.layers.Flatten()(f3)
    
    f4 = tf.keras.layers.Concatenate(1)([f2,f3])
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################

    ## Fully connected Layers

    y = tf.keras.layers.Concatenate()([x4,f4])
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=[input1, input2], outputs=y)
    
    optimizer_function = tf.keras.optimizers.Adagrad(learning_rate=learn_rate)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = optimizer_function, loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = optimizer_function, loss = loss)

    return model

In [ ]:
DLNN_CORENup_RNAFold().summary()

In [ ]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Model" : [],
    "Dataset" : [],
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

for root, dirs, files in os.walk(input_data_folder):
    for file in files:
        
        input_data_file = os.path.join(root, file)
        
        current_dataset_variety = input_data_file.split("\\")[-1].split(".")[0]

        csv_data = pd.read_csv(input_data_file)

        ##################################################################################
        ##### extract data from the current CSV file
        ##################################################################################

        csv_data["OHE_Sequence"] = pd.Series([one_hot_encode_rna(val) for val in csv_data["Sequence"]])
        csv_data["OHE_RNAfold"] = pd.Series([one_hot_encode_rnafold(val) for val in csv_data["RNAfold"]])

        df_positive = csv_data[csv_data['Number'].str.contains("P")]
        df_negative = csv_data[csv_data['Number'].str.contains("N")]

        positive_onehotencoded_List = (np.array(list(df_positive['OHE_Sequence'])), np.array(list(df_positive['OHE_RNAfold'])))
        negative_onehotencoded_List = (np.array(list(df_negative['OHE_Sequence'])), np.array(list(df_negative['OHE_RNAfold'])))

        print("\n======================================================================")
        print("\nFile:", input_data_file)
        print("Positive:", len(positive_onehotencoded_List[0]))
        print("Negative:", len(negative_onehotencoded_List[0]))

        ##################################################################################
        ##### Generate Folds from dataset, and store to file
        ##################################################################################

        ## create the features and labels datasets for the training
        input_seq_len = len(positive_onehotencoded_List[0][0])
        input_size_1 = (len(positive_onehotencoded_List[0][0]), 4)
        input_size_2 = (len(positive_onehotencoded_List[0][0]), 3)

        labels = np.concatenate((np.ones((positive_onehotencoded_List[0].shape[0], 1), 
                                         dtype=np.float32), 
                                 np.zeros((negative_onehotencoded_List[0].shape[0], 1), 
                                          dtype=np.float32)), 
                                axis=0)
        features_1 = np.concatenate((positive_onehotencoded_List[0], 
                                     negative_onehotencoded_List[0]), 
                                    axis=0)
        features_2 = np.concatenate((positive_onehotencoded_List[1], 
                                     negative_onehotencoded_List[1]), 
                                    axis=0)

        folds = build_kfold_multifeature(features_1, features_2, labels, 
                                         k=n_fold, shuffle=shuffle, seed=seed)

        ## Write the k-fold dataset to file
        foldPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold))
        if(not os.path.isdir(foldPath)):
            os.makedirs(foldPath)
        pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

        ## Create and set directory to save model
        modelPath = os.path.join(outPath, expName, current_dataset_variety, "{}fold".format(n_fold), "models")
        if(not os.path.isdir(modelPath)):
            os.makedirs(modelPath)

        ##################################################################################
        ##### TRAIN and PREDICT for every Fold, using models
        ##################################################################################

        # fold counter
        i = 0

        for fold in folds:

            print("\nTrain/Test model "+current_dataset_variety+" on Fold #"+str(i)+".")

            model = DLNN_CORENup_RNAFold(input_seq_len = input_seq_len)

            # Define the model callbacks for early stopping and saving the model. Then train model
            modelCallbacks = [
                tf.keras.callbacks.ModelCheckpoint(os.path.join(modelPath, "{}_bestModel-fold{}.hdf5".format(current_dataset_variety, i)),
                                                   monitor = 'val_loss', verbose = 1, save_best_only = True, 
                                                   save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
            ]
            
            # train the model
            model.fit(x = [fold["X1_train"], fold["X2_train"]], y = fold["y_train"], 
                      batch_size = batch_size, epochs = epochs, verbose = 1, 
                      callbacks = modelCallbacks, validation_split=0.2)

            ##################################################################################
            ##### Prediction and metrics for TRAIN dataset
            ##################################################################################

            y_pred = model.predict([fold["X1_train"], fold["X2_train"]])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_train"], label_pred)
            prec = precision_score(fold["y_train"],label_pred)

            conf = confusion_matrix(fold["y_train"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
            auc = roc_auc_score(fold["y_train"], y_pred)

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Train")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            ##################################################################################
            ##### Prediction and metrics for TEST dataset
            ##################################################################################

            y_pred = model.predict([fold["X1_test"], fold["X2_test"]])
            label_pred = pred2label(y_pred)
            # Compute precision, recall, sensitivity, specifity, mcc
            acc = accuracy_score(fold["y_test"], label_pred)
            prec = precision_score(fold["y_test"],label_pred)

            conf = confusion_matrix(fold["y_test"], label_pred)
            if(conf[0][0]+conf[1][0]):
                sens = float(conf[0][0])/float(conf[0][0]+conf[1][0])
            else:
                sens = 0.0
            if(conf[1][1]+conf[0][1]):
                spec = float(conf[1][1])/float(conf[1][1]+conf[0][1])
            else:
                spec = 0.0
            if((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0])):
                mcc = (float(conf[0][0])*float(conf[1][1]) - float(conf[1][0])*float(conf[0][1]))/math.sqrt((conf[0][0]+conf[0][1])*(conf[0][0]+conf[1][0])*(conf[1][1]+conf[0][1])*(conf[1][1]+conf[1][0]))
            else:
                mcc= 0.0
            fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
            auc = roc_auc_score(fold["y_test"], y_pred)

            evaluations["Model"].append(current_dataset_variety)
            evaluations["Dataset"].append(current_dataset_variety)
            evaluations["Fold"].append(i)
            evaluations["Train_Test"].append("Test")
            evaluations["Accuracy"].append(acc)
            evaluations["Precision"].append(prec)
            evaluations["TPR"].append(tpr)
            evaluations["FPR"].append(fpr)
            evaluations["TPR_FPR_Thresholds"].append(thresholds)
            evaluations["AUC"].append(auc)
            evaluations["Sensitivity"].append(sens)
            evaluations["Specificity"].append(spec)
            evaluations["MCC"].append(mcc)

            i = i+1
            del model
            tf.keras.backend.clear_session()

        ##################################################################################
        ##### Dump evaluations to a file
        ##################################################################################

        evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
        if(not os.path.isdir(evalPath)):
            os.makedirs(evalPath)

        pickle.dump(evaluations,
                    open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "wb"))

## Visualization of Evaluation

In [ ]:
##################################################################################
##### Add import statement here, to make this next part of code standalone executable
##################################################################################

import os
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import numpy as np
import pandas as pd


In [ ]:
##################################################################################
##### Load file and convert to dataframe for easy manipulation
##################################################################################

evalPath = os.path.join(outPath, expName, "_Evaluation_All_Datasets")
if(not os.path.isdir(evalPath)):
    os.makedirs(evalPath)

evaluations = pickle.load(open(os.path.join(evalPath, "{}fold_evaluations.pickle".format(n_fold)), "rb"))

In [ ]:
# evaluations["Model"] = evaluations["Model"][0:20]
# evaluations_df = pd.DataFrame.from_dict(evaluations)

In [ ]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

##################################################################################
##### Group dataset (mean of metrics) by [Dataset, Model, Train_Test] combinations
##################################################################################

evaluations_df_grouped = evaluations_df.groupby(["Dataset", 
                                                 "Model", 
                                                 "Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

# DLNN_3 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_3'])]
# DLNN_5 = evaluations_df_grouped[np.in1d(evaluations_df_grouped.index.get_level_values(1), ['DLNN_5'])]

# DLNN_3_Train = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Train'])]
# DLNN_3_Test = DLNN_3[np.in1d(DLNN_3.index.get_level_values(2), ['Test'])]

# DLNN_5_Train = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Train'])]
# DLNN_5_Test = DLNN_5[np.in1d(DLNN_5.index.get_level_values(2), ['Test'])]

In [ ]:
evaluations_df_grouped